In [16]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [17]:
function kalman_filter_graph()
    x_prev_add = AdditionNode()
    
    x_prior = datavar(:x_prior, Normal{Float64})
    add_1   = constvar(:add_1, 1.0)

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    
    add_x_and_noise = AdditionNode()
    
    x = simplerandomvar(:x)
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = datavar(:y, Float64)
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(x_prev_add)
    activate!(add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getbelief(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    beliefs = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getbelief(x) |> enumerate(), (t) -> beliefs[t[1]] = getdata(t[2]))
    
    for (i, d) in enumerate(data)
        update!(y, d)
    end
    
    return beliefs
end

kalman (generic function with 1 method)

In [18]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [21]:
@btime kalman($data)

  991.695 μs (16107 allocations: 695.03 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-5.188395614670568, σ=14.002800840280099)
 Normal{Float64}(μ=-19.082070237806146, σ=9.950371902099892)
 Normal{Float64}(μ=-13.808216050673948, σ=8.137884587711596)
 Normal{Float64}(μ=-10.554880084897011, σ=7.053456158585984)
 Normal{Float64}(μ=-12.901889285345822, σ=6.311944030978033)
 Normal{Float64}(μ=-10.91455520839113, σ=5.763904177042351)
 Normal{Float64}(μ=-8.346871267484092, σ=5.337605126836238)
 Normal{Float64}(μ=-3.256087604066618, σ=4.993761694389224)
 Normal{Float64}(μ=-1.1574144238629687, σ=4.708816093480111)
 Normal{Float64}(μ=0.7892327857948981, σ=4.467670516087703)
 Normal{Float64}(μ=2.4527644959816914, σ=4.2601432284230505)
 Normal{Float64}(μ=4.926239641815989, σ=4.079085082240021)
 Normal{Float64}(μ=7.64573103021738, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=590.2243784572985, σ=0.582706653680032)
 Normal{Float64}(μ=591.2481571671901, σ=0.5822126418123303)
 Normal{Float64}(μ=592.2611980054527, σ=0.58171988427037

In [20]:
@time kalman(data)

  0.001676 seconds (16.11 k allocations: 695.031 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-5.188395614670568, σ=14.002800840280099)
 Normal{Float64}(μ=-19.082070237806146, σ=9.950371902099892)
 Normal{Float64}(μ=-13.808216050673948, σ=8.137884587711596)
 Normal{Float64}(μ=-10.554880084897011, σ=7.053456158585984)
 Normal{Float64}(μ=-12.901889285345822, σ=6.311944030978033)
 Normal{Float64}(μ=-10.91455520839113, σ=5.763904177042351)
 Normal{Float64}(μ=-8.346871267484092, σ=5.337605126836238)
 Normal{Float64}(μ=-3.256087604066618, σ=4.993761694389224)
 Normal{Float64}(μ=-1.1574144238629687, σ=4.708816093480111)
 Normal{Float64}(μ=0.7892327857948981, σ=4.467670516087703)
 Normal{Float64}(μ=2.4527644959816914, σ=4.2601432284230505)
 Normal{Float64}(μ=4.926239641815989, σ=4.079085082240021)
 Normal{Float64}(μ=7.64573103021738, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=590.2243784572985, σ=0.582706653680032)
 Normal{Float64}(μ=591.2481571671901, σ=0.5822126418123303)
 Normal{Float64}(μ=592.2611980054527, σ=0.58171988427037

In [19]:
@time kalman(data)

  0.230962 seconds (526.50 k allocations: 26.834 MiB, 5.55% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-5.188395614670568, σ=14.002800840280099)
 Normal{Float64}(μ=-19.082070237806146, σ=9.950371902099892)
 Normal{Float64}(μ=-13.808216050673948, σ=8.137884587711596)
 Normal{Float64}(μ=-10.554880084897011, σ=7.053456158585984)
 Normal{Float64}(μ=-12.901889285345822, σ=6.311944030978033)
 Normal{Float64}(μ=-10.91455520839113, σ=5.763904177042351)
 Normal{Float64}(μ=-8.346871267484092, σ=5.337605126836238)
 Normal{Float64}(μ=-3.256087604066618, σ=4.993761694389224)
 Normal{Float64}(μ=-1.1574144238629687, σ=4.708816093480111)
 Normal{Float64}(μ=0.7892327857948981, σ=4.467670516087703)
 Normal{Float64}(μ=2.4527644959816914, σ=4.2601432284230505)
 Normal{Float64}(μ=4.926239641815989, σ=4.079085082240021)
 Normal{Float64}(μ=7.64573103021738, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=590.2243784572985, σ=0.582706653680032)
 Normal{Float64}(μ=591.2481571671901, σ=0.5822126418123303)
 Normal{Float64}(μ=592.2611980054527, σ=0.58171988427037